<a href="https://colab.research.google.com/github/sourcecode369/deep-natural-language-processing/blob/master/memory%20networks/Memory_Networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import absolute_import, print_function, unicode_literals, division
from builtins import range, input

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import re 
import os
import sys
import gc
gc.enable()
import tarfile

from keras.models import Model, Sequential
from keras.layers import Dense, Embedding, Input, Lambda, Reshape, add, dot, Activation 
from keras.preprocessing.sequence import pad_sequences
from keras.optimizers import Adam, RMSprop
from keras.utils import get_file
import keras.backend as K

import warnings
warnings.simplefilter("ignore")
warnings.filterwarnings("ignore")

Using TensorFlow backend.


### Single Supproting Fact

In [0]:
path = get_file('babi-tasks-v1-2.tar.gz',
                origin='https://s3.amazonaws.com/text-datasets/babi_tasks_1-20_v1-2.tar.gz')

tar = tarfile.open(path)

In [0]:
challenges = {
    'single_supporting_fact_10k':'tasks_1-20_v1-2/en-10k/qa1_single-supporting-fact_{}.txt',
    'two_supporting_fact_10k':'tasks_1-20_v1-2/en-10k/qa2_two-supporting-facts_{}.txt'
}

In [0]:
def tokenize(sent):
  return [x.strip() for x in re.split('(\W+)?',sent) if x.strip()]

In [0]:
def get_stories(f):
  data = []
  story = []
  printed = False
  count = 0
  for line in f:
    count+=1
    if count < 5:
      print(line)
    line = line.decode('utf-8').strip()
    nid, line = line.split(' ', 1)
    if int(nid) == 1:
      story = []
    if '\t' in line:
      q, a, supporting = line.split('\t')
      q = tokenize(q)
      story_so_far = [[str(i)] + s for i, s in enumerate(story) if s]
      data.append((story_so_far, q, a))
      story.append('')
    else:
      story.append(tokenize(line))
  return data

In [0]:
def should_flatten(el):
  return not isinstance(el, (str, bytes))

def flatten(l):
  for el in l:
    if should_flatten(el):
      yield from flatten(el)
    else:
      yield el

In [0]:
def vectorize_stories(data, word2idx, story_maxlen, query_maxlen):
  inputs, queries, answers = [], [], []
  for story, query, answer in data:
    inputs.append([[word2idx[w] for w in s] for s in story])
    queries.append([word2idx[w] for w in query])
    answers.append([word2idx[answer]])
  return (
    [pad_sequences(x, maxlen=story_maxlen) for x in inputs],
    pad_sequences(queries, maxlen=query_maxlen),
    np.array(answers)
  )

In [0]:
def stack_inputs(inputs, story_maxsents, story_maxlen):
  for i, story in enumerate(inputs):
    inputs[i] = np.concatenate(
        [
         story, 
         np.zeros((story_maxsents-story.shape[0], story_maxlen),'int')
        ]
    )
  return np.stack(inputs)

In [0]:
def get_data(challenge_type):
  challenge = challenges[challenge_type]
  
  train_stories = get_stories(tar.extractfile(challenge.format('train')))
  test_stories = get_stories(tar.extractfile(challenge.format('test')))
  
  stories = train_stories + test_stories
  
  story_maxlen = max((len(s) for x, _, _ in stories for s in x))
  story_maxsents = max((len(x) for x, _, _ in stories))
  query_maxlen = max(len(x) for _, x, _ in stories)

  vocab = sorted(set(flatten(stories)))
  vocab.insert(0, '<PAD>')
  vocab_size = len(vocab)

  word2idx = {c:i for i, c in enumerate(vocab)}

  inputs_train, queries_train, answers_train = vectorize_stories(
      train_stories,
      word2idx,
      story_maxlen,
      query_maxlen
  )
  inputs_test, queries_test, answers_test = vectorize_stories(
      test_stories, 
      word2idx,
      story_maxlen,
      query_maxlen
  )
  inputs_train = stack_inputs(inputs_train, story_maxsents, story_maxlen)
  inputs_test = stack_inputs(inputs_test, story_maxsents, story_maxlen)
  print(f"inputs_train.shape {inputs_train.shape}, inputs_test.shape {inputs_test.shape}")
  return train_stories, test_stories, inputs_train, queries_train, answers_train, \
  inputs_test, queries_test, answers_test, story_maxsents, story_maxlen, query_maxlen, vocab, vocab_size 

In [0]:
train_stories, test_stories, inputs_train, queries_train, answers_train, \
  inputs_test, queries_test, answers_test, story_maxsents, story_maxlen, query_maxlen, vocab, vocab_size = get_data('single_supporting_fact_10k')

b'1 Mary moved to the bathroom.\n'
b'2 John went to the hallway.\n'
b'3 Where is Mary? \tbathroom\t1\n'
b'4 Daniel went back to the hallway.\n'
b'1 John travelled to the hallway.\n'
b'2 Mary journeyed to the bathroom.\n'
b'3 Where is John? \thallway\t1\n'
b'4 Daniel went back to the bathroom.\n'
inputs_train.shape (10000, 10, 8), inputs_test.shape (1000, 10, 8)


In [0]:
embedding_dim = 15

input_story_ = Input((story_maxsents, story_maxlen))
embedded_story = Embedding(vocab_size, embedding_dim)(input_story_)
embedded_story = Lambda(lambda x: K.sum(x, axis=2))(embedded_story)
print('input_story_.shape, embedded_story.shape: ', input_story_.shape, embedded_story.shape)


input_question_ = Input((query_maxlen, ))
embedded_question = Embedding(vocab_size, embedding_dim)(input_question_)
embedded_question = Lambda(lambda x: K.sum(x, axis=1))(embedded_question)

embedded_question = Reshape((1, embedding_dim))(embedded_question)
print('inp_q.shape, emb_q.shape', input_question_.shape, embedded_question.shape)

x = dot([embedded_story, embedded_question], 2)
x = Reshape((story_maxsents, ))(x)
x = Activation('softmax')(x)
story_weights = Reshape((story_maxsents, 1))(x)
print("story_weights.shape", story_weights.shape)

x = dot([story_weights, embedded_story], 1)
x = Reshape((embedding_dim, ))(x)
ans = Dense(vocab_size, activation='softmax')(x)

model = Model([input_story_, input_question_], ans)

model.compile(optimizer = RMSprop(lr=1e-2),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

r = model.fit([inputs_train, queries_train],
              answers_train,
              epochs=10,
              batch_size=32,
              validation_data=([inputs_test, queries_test], answers_test)
             )

input_story_.shape, embedded_story.shape:  (?, 10, 8) (?, 10, 15)
inp_q.shape, emb_q.shape (?, 4) (?, 1, 15)
story_weights.shape (?, 10, 1)


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 10000 samples, validate on 1000 samples
Epoch 1/10





10000/10000 [==============================] - 9s 879us/step - loss: 0.7462 - acc: 0.7281 - val_loss: 0.0239 - val_acc: 0.9950
Epoch 2/10
10000/10000 [==============================] - 1s 147us/step - loss: 0.0122 - acc: 0.9962 - val_loss: 1.9996e-04 - val_acc: 1.0000
Epoch 3/10
10000/10000 [==============================] - 1s 146us/step - loss: 0.0049 - acc: 0.9990 - val_loss: 0.0051 - val_acc: 0.9960
Epoch 4/10
10000/10000 [==============================] - 1s 145us/step - loss: 0.0034 - acc: 0.9991 - val_loss: 0.0037 - val_acc: 0.9990
Epoch 5/10
10000/10000 [==============================] - 2s 151us/step - loss: 0.0040 - acc: 0.9990 - val_loss: 3.2651e-06 - val_acc: 1.0000
Epoch 6/1

#### Demo

In [0]:
while True:

  debug_model = Model([input_story_, input_question_], story_weights)

  story_idx = np.random.choice(len(train_stories))

  i = inputs_train[story_idx:story_idx+1]
  q = queries_train[story_idx:story_idx+1]
  w = debug_model.predict([i,q]).flatten()

  story, question, ans = train_stories[story_idx]
  print("story:\n")
  for i, line in enumerate(story):
    print("{:1.5f}".format(w[i]), "\t", " ".join(line))
  print()
  print("question: ", " ".join(question))
  print("answer: ", ans)

  print()
  if input("Another story.? y/n") == "n":
    break

story:

0.00000 	 0 John moved to the bedroom .
0.00000 	 1 Daniel journeyed to the bathroom .
0.00000 	 3 Daniel moved to the hallway .
0.00000 	 4 Sandra journeyed to the garden .
0.00010 	 6 Daniel went back to the bedroom .
0.00000 	 7 Mary moved to the hallway .
0.02529 	 9 Daniel went to the kitchen .
0.97460 	 10 Daniel went back to the hallway .
0.00000 	 12 Sandra went to the bathroom .
0.00001 	 13 Sandra travelled to the bedroom .

question:  Where is Daniel ?
answer:  hallway

Another story.? y/ny
story:

0.00000 	 0 Daniel went to the bedroom .
0.00000 	 1 Daniel travelled to the office .
0.00000 	 3 Sandra went to the office .
0.00000 	 4 John travelled to the office .
0.00000 	 6 John travelled to the kitchen .
0.00000 	 7 John journeyed to the office .
0.04168 	 9 Daniel moved to the bathroom .
0.95832 	 10 Daniel moved to the garden .

question:  Where is Daniel ?
answer:  garden

Another story.? y/ny
story:

0.00000 	 0 Mary moved to the hallway .
0.00000 	 1 Mary tra